# Introduction

*Schistosoma mansoni* are parasitic worms that cause chronic schistosomiasis. Once inside the host, they trigger an immune response that leads to large tissue granulomas, causing fibrosis, organomegaly, and tissue damage.

*S. mansoni* have a complex life cycle. Eggs from humans exit into the water where they hatch into miracidia which then infect a snail. While in the snail, sporocysts develop and eventually turn into cercariae and are released back into the water. The cercariae then penatrate the skin of human or mouse and become adults and make new eggs, starting the process over. Some portion of the eggs don't go through the intestine and exit the human, but instead can end up in other areas, particularly the liver. The eggs that make it to the intestines continue their life cycle, while the others are trapped and eventually die.

Since most of our previous knowledge is based on studies looking at *S. mansoni* from eggs trapped in the liver instead of the eggs that move through the intestines, we want to understand the differences, if there are any, between miracidia from the liver versus the intestines. A past study was done that determined that was a significant difference in gene expression between eggs isolated from the liver versus the intestines. Our goal is to see if the miracidia also exhibit a significant difference in gene expression between those stuck in the liver and those that make it to the intestines.

# Methods

## RNA extraction, library preparation, and sequencing

Will be written by Dr.Wheeler.

## Read QC and alignment

First, we ran quality control on our raw reads using FastQC (v0.12.1). The reads were then trimmed using Cutadapt (v4.9) with arguments to remove the first 10 base pairs that are the barcode, to remove both the forwared and reverse adapters, and to deal with the incorrect "G" calls due to NextSeq Illumina sequencing. We then reran FastQC to double check the quality after trimming. Reads were then mapped to the *S. mansoni* genome (version WBPS19 from WormBase ParaSite) using STAR (v2.7.10b). Two alignments were done. Qualimap (v2.3) was use to check the quality control of the alignment sequencing data from STAR. Duplicate reads were tagged using MarkDuplicates (v3.2.0). Reads were then assigned to features such as genes, exons, promotors, etc. and counts the numbers of reads assigned to each feature using FeatureCounts (v2.0.6). Lastly, a HTML report of the quality controls was produced using MultiQC (v1.17).

## Identification of differentially expressed genes and GO analysis

Count matrix of reads were filtered to remove reads with a count of 0 and reformatted using R. The reformatted matrix was then used as input for DESeq2 (v1.46.0). The results of DESeq2 were then used to identify significantlly differentially expressed genes (abs(log2FoldChange) > 2 and padj <= 0.05). These genes were then used as input for gProfiler2 (v0.2.3) for functional enrichment analysis.

# Results

## RNA sequencing, alignment, and sample clustering

## Identification of differentially expressed genes

## Identification of pathways and/or GO annotations that are differentially expressed

# Discussion

# References

**Change format of referecnces listed below** 
   
1. Peterková K, Konečný L, Macháček T et al. Winners vs. losers: Schistosoma mansoni intestinal and liver eggs exhibit striking differences in gene expression and immunogenicity. 2024.
2. Hudak et al. Open OnDemand: A web-based client portal for HPC centers. Journal of Open Source Software. 3(25), 622. 2018.
3. Saber S, Alomar S, Yahya G. Blocking prostanoid receptors switches on multiple immune responses and cascades of inflammatory signaling against larval stages in snail fever. Environmental Science and Pollution Research. 2022.
4. Andrew S. FastQC: A Quality Control Tool for High Throughput Sequence Data. Babraham Bioinformatics. 2010.
5. Martin M. Cutadapt removes adapter sequences from high-throughput sequencing reads. EMBnet Stichting.
6. Dobin A, Davis C, Schlesinger F et al. STAR: ultrafast universal RNA-seq aligner. Bioinformatics. 2013.
7. Okonechnikov K, Conesa A, García-Alcalde F. Qualimap 2: advanced multi-sample quality control for high-throughput sequencing data." Bioinformatics. 2015.
8. Picard toolkit. Broad Institute. 2019.
9. Liao Y, Smyth G, Shi W. featureCounts: an efficient general purpose program for assigning sequence reads to genomic features. Bioinformatics. 2014.
10. Ewels P, Magnusson M, Lundin S et al. MultiQC: summarize analysis results for multiple tools and samples in a single report. Bioinformatics, 2016.
11. Love MI, Love MI, Huber W et al. Moderated estimation of fold change and dispersion for RNA-seq data with DESeq2. BioMed Central.